In [ ]:
using WAV;
using Gen;
using Printf;
import Random;
include("./model.jl");
include("./plotting.jl");
include("./proposals.jl");
include("./proposals.jl");
include("./gammatonegram.jl");
include("./time_helpers.jl");
include("./inference_helpers.jl");
include("./initialization.jl");

In [ ]:
source_params=Dict("types"=>["noise","tone"], "n_harmonics"=>30,"sources"=>Dict("type"=>"max","val"=>5),
    "elements"=>Dict("type"=>"max","val"=>10,
        "wait"=>Dict("a"=>Dict("a"=>20,"b"=>1),"mu"=>Dict("mu"=>-2.0,"sig"=>1.5)),
        "dur"=>Dict("a"=>Dict("a"=>20,"b"=>1),"mu"=>Dict("mu"=>-2.0,"sig"=>1.5))),
    "dream"=>true)   
audio_sr=20000; steps = Dict("t"=>0.020, "min"=> 0.020, "f"=>4); 
gtg_params=Dict("twin"=>0.025, "thop"=>0.010, "nfilts"=>64, "fmin"=>50, "width"=>0.50, 
    "log_constant"=>1e-80, "dB_threshold"=>20, "ref"=>1e-6, "plot_max"=>90)
wts, f = gtg_weights(audio_sr, gtg_params)
scene_duration = 0.5
obs_noise = Dict("type"=>"fixed","val"=>1.0)
args = (source_params, float(scene_duration), wts, steps, Int(audio_sr), obs_noise, gtg_params);

In [ ]:
tre = []
for trace_idx = 1
    tr, _ = generate(generate_scene, args)
    scene_gram, t, scene_wave, source_waves, element_waves = get_retval(tr)
    println(size(element_waves))
    n_elements = []
    for i = 1:length(element_waves)
        for j = 1:length(element_waves[i])
            gtg, t = gammatonegram(element_waves[i][j],wts,audio_sr,gtg_params)
            push!(n_elements, all(gtg .== gtg_params["dB_threshold"]) ? 0 : 1)
        end
    end
    push!(tre, sum(n_elements))
end
    


In [ ]:
wavwrite(scene_wave/maximum(abs.(scene_wave)),"scene.wav",Fs=audio_sr)

In [ ]:
(fwd_choices, fwd_score, fwd_ret) = propose(type_randomness, (tr,1,))
(new_trace, bwd_choices, weight) = type_involution(tr, fwd_choices, fwd_ret, ())
(bwd_score, _) = assess(type_randomness, (new_trace,1,), bwd_choices)
println((weight - fwd_score + bwd_score))
println("Weight: $(weight), Fwd: $(fwd_score), Bwd: $(bwd_score)")
# trace, accepted = mh(trace, switch_randomness, (), switch_involution) 
plot_sources(new_trace, scene_gram, 1)
println(fwd_choices)
println("Backward choices")
println(bwd_choices)

In [ ]:
scene_gram, t, scene_wave, = get_retval(new_trace)
plot_gtg(scene_gram, scene_duration, audio_sr/2., 20, 100)